In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms, datasets, models
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Define the path to your dataset
path = '/home/rishabh.mondal/R_P_project/Faulty_solar_panel'

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=path, transform=transform)

# Define the number of splits for cross-validation
n_splits = 4  # 4-fold cross-validation

# Initialize stratified k-fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [3]:
import torchvision.models as models

# Load pre-trained DenseNet model (DenseNet-121 in this case)
model = models.densenet121(pretrained=True)

# Print the model architecture
print(model)



/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [4]:
# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))

    # Define the Resnet18 model
    model = models.densenet121(pretrained=True)
    model.classifier = nn.Linear(1024, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        if epoch % 10 == 9:  # Print results every 10 epochs
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                val_losses.append(val_loss / len(val_loader))
                accuracy = correct / total
                accuracies.append(accuracy)

                # Calculate precision, recall, and F1 score
                precision = precision_score(all_labels, all_predicted, average='weighted')
                recall = recall_score(all_labels, all_predicted, average='weighted')
                f1 = f1_score(all_labels, all_predicted, average='weighted')

                print(f'Fold: {fold + 1}, Epoch {epoch + 1}/{num_epochs}, '
                      f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
                      f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}, '
                      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # # Plotting loss and accuracy curves
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(train_losses, label='Train Loss')
    # plt.plot(val_losses, label='Validation Loss')
    # plt.xlabel('Epochs')
    # plt


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 1, Epoch 10/50, Train Loss: 0.0138, Train Accuracy: 0.9970, Val Loss: 0.5419, Val Accuracy: 0.8468, Precision: 0.8549, Recall: 0.8468, F1 Score: 0.8460
Fold: 1, Epoch 20/50, Train Loss: 0.0730, Train Accuracy: 0.9894, Val Loss: 1.8973, Val Accuracy: 0.6532, Precision: 0.7613, Recall: 0.6532, F1 Score: 0.6561
Fold: 1, Epoch 30/50, Train Loss: 0.0091, Train Accuracy: 0.9955, Val Loss: 0.5102, Val Accuracy: 0.8739, Precision: 0.8808, Recall: 0.8739, F1 Score: 0.8725
Fold: 1, Epoch 40/50, Train Loss: 0.0057, Train Accuracy: 0.9970, Val Loss: 0.5241, Val Accuracy: 0.8829, Precision: 0.8878, Recall: 0.8829, F1 Score: 0.8820
Fold: 1, Epoch 50/50, Train Loss: 0.0119, Train Accuracy: 0.9955, Val Loss: 1.4788, Val Accuracy: 0.7387, Precision: 0.7978, Recall: 0.7387, F1 Score: 0.7267
Training Fold 2/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 2, Epoch 10/50, Train Loss: 0.0251, Train Accuracy: 0.9955, Val Loss: 1.1049, Val Accuracy: 0.7873, Precision: 0.8143, Recall: 0.7873, F1 Score: 0.7823
Fold: 2, Epoch 20/50, Train Loss: 0.0777, Train Accuracy: 0.9699, Val Loss: 1.1245, Val Accuracy: 0.7647, Precision: 0.7811, Recall: 0.7647, F1 Score: 0.7585
Fold: 2, Epoch 30/50, Train Loss: 0.0220, Train Accuracy: 0.9925, Val Loss: 0.8249, Val Accuracy: 0.8281, Precision: 0.8454, Recall: 0.8281, F1 Score: 0.8275
Fold: 2, Epoch 40/50, Train Loss: 0.0038, Train Accuracy: 1.0000, Val Loss: 0.7465, Val Accuracy: 0.8371, Precision: 0.8440, Recall: 0.8371, F1 Score: 0.8343
Fold: 2, Epoch 50/50, Train Loss: 0.0004, Train Accuracy: 1.0000, Val Loss: 0.8205, Val Accuracy: 0.8597, Precision: 0.8663, Recall: 0.8597, F1 Score: 0.8594
Training Fold 3/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 3, Epoch 10/50, Train Loss: 0.0260, Train Accuracy: 0.9940, Val Loss: 0.6031, Val Accuracy: 0.8371, Precision: 0.8464, Recall: 0.8371, F1 Score: 0.8371
Fold: 3, Epoch 20/50, Train Loss: 0.0124, Train Accuracy: 0.9955, Val Loss: 0.9987, Val Accuracy: 0.7828, Precision: 0.8176, Recall: 0.7828, F1 Score: 0.7856
Fold: 3, Epoch 30/50, Train Loss: 0.0154, Train Accuracy: 0.9925, Val Loss: 0.7151, Val Accuracy: 0.8462, Precision: 0.8556, Recall: 0.8462, F1 Score: 0.8463
Fold: 3, Epoch 40/50, Train Loss: 0.0268, Train Accuracy: 0.9940, Val Loss: 1.1329, Val Accuracy: 0.7421, Precision: 0.7789, Recall: 0.7421, F1 Score: 0.7426
Fold: 3, Epoch 50/50, Train Loss: 0.0099, Train Accuracy: 0.9985, Val Loss: 0.7981, Val Accuracy: 0.8643, Precision: 0.8684, Recall: 0.8643, F1 Score: 0.8651
Training Fold 4/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 4, Epoch 10/50, Train Loss: 0.0116, Train Accuracy: 0.9955, Val Loss: 0.8781, Val Accuracy: 0.8190, Precision: 0.8400, Recall: 0.8190, F1 Score: 0.8169
Fold: 4, Epoch 20/50, Train Loss: 0.0368, Train Accuracy: 0.9880, Val Loss: 0.7787, Val Accuracy: 0.8462, Precision: 0.8646, Recall: 0.8462, F1 Score: 0.8445
Fold: 4, Epoch 30/50, Train Loss: 0.0065, Train Accuracy: 0.9985, Val Loss: 0.6152, Val Accuracy: 0.8824, Precision: 0.8857, Recall: 0.8824, F1 Score: 0.8824
Fold: 4, Epoch 40/50, Train Loss: 0.0049, Train Accuracy: 0.9985, Val Loss: 0.6028, Val Accuracy: 0.8959, Precision: 0.8963, Recall: 0.8959, F1 Score: 0.8952
Fold: 4, Epoch 50/50, Train Loss: 0.0037, Train Accuracy: 0.9985, Val Loss: 0.5688, Val Accuracy: 0.8869, Precision: 0.8885, Recall: 0.8869, F1 Score: 0.8861


In [5]:
# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))

    # Define the ResNet18 model
    model = models.densenet121(pretrained=False)
    model.classifier = nn.Linear(1024, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        if epoch % 10 == 9:  # Print results every 10 epochs
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                val_losses.append(val_loss / len(val_loader))
                accuracy = correct / total
                accuracies.append(accuracy)

                # Calculate precision, recall, and F1 score
                precision = precision_score(all_labels, all_predicted, average='weighted')
                recall = recall_score(all_labels, all_predicted, average='weighted')
                f1 = f1_score(all_labels, all_predicted, average='weighted')

                print(f'Fold: {fold + 1}, Epoch {epoch + 1}/{num_epochs}, '
                      f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
                      f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}, '
                      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # # Plotting loss and accuracy curves
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(train_losses, label='Train Loss')
    # plt.plot(val_losses, label='Validation Loss')
    # plt.xlabel('Epochs')
    # plt


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Fold: 1, Epoch 10/50, Train Loss: 0.2242, Train Accuracy: 0.9321, Val Loss: 1.5707, Val Accuracy: 0.5631, Precision: 0.5526, Recall: 0.5631, F1 Score: 0.5251
Fold: 1, Epoch 20/50, Train Loss: 0.0157, Train Accuracy: 0.9970, Val Loss: 1.3178, Val Accuracy: 0.6712, Precision: 0.6921, Recall: 0.6712, F1 Score: 0.6701
Fold: 1, Epoch 30/50, Train Loss: 0.0400, Train Accuracy: 0.9910, Val Loss: 1.4081, Val Accuracy: 0.7027, Precision: 0.7366, Recall: 0.7027, F1 Score: 0.7011
Fold: 1, Epoch 40/50, Train Loss: 0.1492, Train Accuracy: 0.9623, Val Loss: 3.3249, Val Accuracy: 0.5000, Precision: 0.6276, Recall: 0.5000, F1 Score: 0.5137
Fold: 1, Epoch 50/50, Train Loss: 0.0386, Train Accuracy: 0.9910, Val Loss: 1.5937, Val Accuracy: 0.6532, Precision: 0.6790, Recall: 0.6532, F1 Score: 0.6525
Training Fold 2/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Fold: 2, Epoch 10/50, Train Loss: 0.1764, Train Accuracy: 0.9533, Val Loss: 2.2686, Val Accuracy: 0.4615, Precision: 0.5344, Recall: 0.4615, F1 Score: 0.4099
Fold: 2, Epoch 20/50, Train Loss: 0.0816, Train Accuracy: 0.9834, Val Loss: 1.9144, Val Accuracy: 0.5566, Precision: 0.6892, Recall: 0.5566, F1 Score: 0.5591
Fold: 2, Epoch 30/50, Train Loss: 0.0548, Train Accuracy: 0.9895, Val Loss: 1.9678, Val Accuracy: 0.6380, Precision: 0.6616, Recall: 0.6380, F1 Score: 0.6419
Fold: 2, Epoch 40/50, Train Loss: 0.0476, Train Accuracy: 0.9910, Val Loss: 1.6610, Val Accuracy: 0.6561, Precision: 0.6875, Recall: 0.6561, F1 Score: 0.6428
Fold: 2, Epoch 50/50, Train Loss: 0.0652, Train Accuracy: 0.9880, Val Loss: 2.1009, Val Accuracy: 0.6290, Precision: 0.6528, Recall: 0.6290, F1 Score: 0.6291
Training Fold 3/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Fold: 3, Epoch 10/50, Train Loss: 0.2422, Train Accuracy: 0.9262, Val Loss: 1.7818, Val Accuracy: 0.5249, Precision: 0.5419, Recall: 0.5249, F1 Score: 0.4897
Fold: 3, Epoch 20/50, Train Loss: 0.0942, Train Accuracy: 0.9729, Val Loss: 1.7882, Val Accuracy: 0.5792, Precision: 0.6486, Recall: 0.5792, F1 Score: 0.5484
Fold: 3, Epoch 30/50, Train Loss: 0.0738, Train Accuracy: 0.9834, Val Loss: 1.4784, Val Accuracy: 0.6606, Precision: 0.7267, Recall: 0.6606, F1 Score: 0.6607
Fold: 3, Epoch 40/50, Train Loss: 0.0908, Train Accuracy: 0.9699, Val Loss: 1.2008, Val Accuracy: 0.7330, Precision: 0.7667, Recall: 0.7330, F1 Score: 0.7354
Fold: 3, Epoch 50/50, Train Loss: 0.0630, Train Accuracy: 0.9819, Val Loss: 1.4071, Val Accuracy: 0.6878, Precision: 0.7248, Recall: 0.6878, F1 Score: 0.6853
Training Fold 4/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Fold: 4, Epoch 10/50, Train Loss: 0.2415, Train Accuracy: 0.9322, Val Loss: 1.4332, Val Accuracy: 0.6109, Precision: 0.6826, Recall: 0.6109, F1 Score: 0.5972
Fold: 4, Epoch 20/50, Train Loss: 0.0561, Train Accuracy: 0.9804, Val Loss: 1.8466, Val Accuracy: 0.6244, Precision: 0.7819, Recall: 0.6244, F1 Score: 0.5965
Fold: 4, Epoch 30/50, Train Loss: 0.0326, Train Accuracy: 0.9955, Val Loss: 1.1419, Val Accuracy: 0.7647, Precision: 0.7669, Recall: 0.7647, F1 Score: 0.7585
Fold: 4, Epoch 40/50, Train Loss: 0.1200, Train Accuracy: 0.9654, Val Loss: 1.6870, Val Accuracy: 0.6471, Precision: 0.7323, Recall: 0.6471, F1 Score: 0.6525
Fold: 4, Epoch 50/50, Train Loss: 0.0207, Train Accuracy: 0.9970, Val Loss: 0.9845, Val Accuracy: 0.7964, Precision: 0.7980, Recall: 0.7964, F1 Score: 0.7961
